In [ ]:
import pandas as pd

df = pd.read_csv("./src/xauusdm15.filtered.csv")

df.shape

In [ ]:
from modules.taframework.taframework import EnhancedTechnicalAnalyzer, create_analyzer, cabr


query = """
Close above EMA_20
RSI_14 below 30
SMA_200 crossed_down Close
"""
